In [1]:
import requests
import pandas as pd

In [2]:
# Coordinates
locations = {
    "Kondapur": (17.4579, 78.3650),
    "Miyapur": (17.5008, 78.3568),
    "Secunderabad": (17.4399, 78.4983),
    "Begumpet": (17.4432, 78.4600)
}

In [3]:
# Variables
weather_vars = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "wind_speed_10m"
]

# Dates
archive_start = "2025-05-26"
today = "2025-05-29"
forecast_end = "2025-06-09"

# Timezone
tz = "Asia%2FKolkata"

In [4]:
# Store all weather data
all_weather = []

for place, (lat, lon) in locations.items():
    print(f"\n📦 Processing {place}...")

    # --- 1. Historical (Archive) ---
    archive_url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={archive_start}&end_date={today}"
        f"&hourly={','.join(weather_vars)}"
        f"&timezone={tz}"
    )
    print(f"🌤 Archive fetch...")
    r1 = requests.get(archive_url)

    if r1.status_code == 200:
        df1 = pd.DataFrame(r1.json()["hourly"])
        df1["location"] = place
    else:
        print(f"❌ Archive failed: {r1.status_code} - {r1.text}")
        df1 = pd.DataFrame()

    # --- 2. Forecast (Future) ---
    forecast_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly={','.join(weather_vars)}"
        f"&start_date=2025-05-30&end_date={forecast_end}"
        f"&timezone={tz}"
    )
    print(f"🌦 Forecast fetch...")
    r2 = requests.get(forecast_url)

    if r2.status_code == 200:
        df2 = pd.DataFrame(r2.json()["hourly"])
        df2["location"] = place
    else:
        print(f"❌ Forecast failed: {r2.status_code} - {r2.text}")
        df2 = pd.DataFrame()

    # Combine both
    combined = pd.concat([df1, df2], ignore_index=True)
    all_weather.append(combined)

# Final DataFra



📦 Processing Kondapur...
🌤 Archive fetch...
🌦 Forecast fetch...

📦 Processing Miyapur...
🌤 Archive fetch...
🌦 Forecast fetch...

📦 Processing Secunderabad...
🌤 Archive fetch...
🌦 Forecast fetch...

📦 Processing Begumpet...
🌤 Archive fetch...
🌦 Forecast fetch...


In [5]:
weather_complete_df = pd.concat(all_weather)

In [6]:
weather_complete_df.head()

,time,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,location
0,2025-05-26T00:00,23.7,90.0,0.0,11.2,Kondapur
1,2025-05-26T01:00,23.5,91.0,0.0,10.5,Kondapur
2,2025-05-26T02:00,23.4,92.0,0.0,9.1,Kondapur
3,2025-05-26T03:00,23.3,92.0,0.0,8.8,Kondapur
4,2025-05-26T04:00,23.4,91.0,0.0,8.7,Kondapur


In [22]:
weather_complete_df['time'] = pd.to_datetime(weather_complete_df['time'])

In [34]:
nan_data = weather_complete_df.loc[weather_complete_df['temperature_2m'].isna(),['time',  'location']]

In [36]:
nan_data['date'] = nan_data['time'].dt.date

In [41]:
nan_data = nan_data[['date','location']].drop_duplicates()

In [42]:
nan_data

,date,location
54,2025-05-28,Kondapur
72,2025-05-29,Kondapur
54,2025-05-28,Miyapur
72,2025-05-29,Miyapur
54,2025-05-28,Secunderabad
72,2025-05-29,Secunderabad
54,2025-05-28,Begumpet
72,2025-05-29,Begumpet


In [24]:
# nan_dates = list(pd.to_datetime(weather_complete_df.loc[weather_complete_df['temperature_2m'].isna(),'time']).dt.date.unique())

In [31]:
# nan_locs = list(weather_complete_df.loc[weather_complete_df['temperature_2m'].isna(),'location'].unique())

In [25]:
# nan_dates

[datetime.date(2025, 5, 28), datetime.date(2025, 5, 29)]

In [32]:
# nan_locs

['Kondapur', 'Miyapur', 'Secunderabad', 'Begumpet']

In [51]:
for date, location in nan_data.values:
    print(date, location)
    weather_complete_df = weather_complete_df[
        ~(
            (weather_complete_df['location'] == location) &
            (weather_complete_df['time'].dt.date == date)
        )
    ]

2025-05-28 Kondapur
2025-05-29 Kondapur
2025-05-28 Miyapur
2025-05-29 Miyapur
2025-05-28 Secunderabad
2025-05-29 Secunderabad
2025-05-28 Begumpet
2025-05-29 Begumpet


In [52]:
weather_complete_df.shape

(1248, 6)

In [53]:
weather_df.shape

(1440, 6)

In [54]:
for date, location in nan_data.values:
    print(date, location)
    print(weather_complete_df[
        (
            (weather_complete_df['location'] == location) &
            (weather_complete_df['time'].dt.date == date)
        )
    ])

2025-05-28 Kondapur
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-29 Kondapur
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-28 Miyapur
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-29 Miyapur
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-28 Secunderabad
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-29 Secunderabad
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05-28 Begumpet
Empty DataFrame
Columns: [time, temperature_2m, relative_humidity_2m, precipitation, wind_speed_10m, location]
Index: []
2025-05

In [55]:
def fetch_weather_today(lat, lon, location_name, date):
    # date = "2025-05-29"
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation,windspeed_10m"
        f"&start_date={date}&end_date={date}"
        f"&timezone=Asia%2FKolkata"
    )

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["hourly"])
        df["location"] = location_name
        return df
    else:
        print(f"❌ Failed for {location_name}: {response.status_code} - {response.text}")
        return None

In [57]:
for date, location in nan_data.values:
    print(date, location)
    df_interim = fetch_weather_today(17.443, 78.391, location, date)
    weather_complete_df = pd.concat([weather_complete_df, df_interim], ignore_index=True)
    print(weather_complete_df.shape)

2025-05-28 Kondapur
(1272, 7)
2025-05-29 Kondapur
(1296, 7)
2025-05-28 Miyapur
(1320, 7)
2025-05-29 Miyapur
(1344, 7)
2025-05-28 Secunderabad
(1368, 7)
2025-05-29 Secunderabad
(1392, 7)
2025-05-28 Begumpet
(1416, 7)
2025-05-29 Begumpet
(1440, 7)


In [58]:
nan_data2 = weather_complete_df.loc[weather_complete_df['temperature_2m'].isna(),['time',  'location']]

In [59]:
nan_data2 #good lovely thank you god

,time,location


In [60]:
weather_df = weather_complete_df.copy()

In [61]:
# weather_complete_df = weather_df.copy()

### Converting hourly interval into 30 mins intervals

In [62]:
# Ensure 'time' is datetime
weather_df['time'] = pd.to_datetime(weather_df['time'])

In [63]:
# Sort by location and time
weather_df = weather_df.sort_values(['location', 'time']).reset_index(drop=True)

In [64]:
# List of columns to average
cols_to_avg = ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m"]

# Prepare list for new rows
new_rows = []

In [65]:
# Loop through each location separately
for loc, group in weather_df.groupby('location'):
    group = group.sort_values('time').reset_index(drop=True)
    for i in range(len(group) - 1):
        curr_row = group.iloc[i]
        next_row = group.iloc[i + 1]

        # Create new interpolated row
        interpolated_time = curr_row['time'] + pd.Timedelta(minutes=30)
        new_row = {
            'time': interpolated_time,
            'location': loc
        }
        for col in cols_to_avg:
            new_row[col] = (curr_row[col] + next_row[col]) / 2
        new_rows.append(new_row)

# Convert new rows to DataFrame
interpolated_df = pd.DataFrame(new_rows)

In [66]:
interpolated_df.head()

,time,location,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m
0,2025-05-26 00:30:00,Begumpet,24.10,91.5,0.0,8.60
1,2025-05-26 01:30:00,Begumpet,23.90,92.0,0.0,8.25
2,2025-05-26 02:30:00,Begumpet,23.80,92.0,0.0,8.20
3,2025-05-26 03:30:00,Begumpet,23.75,92.0,0.0,8.20
4,2025-05-26 04:30:00,Begumpet,23.85,91.5,0.0,7.35


In [67]:
# Combine original and interpolated
combined_df = pd.concat([weather_df, interpolated_df], ignore_index=True)
combined_df = combined_df.sort_values(['location', 'time']).reset_index(drop=True)

In [68]:
combined_df.head()

,time,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,location,windspeed_10m
0,2025-05-26 00:00:00,24.2,91.0,0.0,8.80,Begumpet,NaN
1,2025-05-26 00:30:00,24.1,91.5,0.0,8.60,Begumpet,NaN
2,2025-05-26 01:00:00,24.0,92.0,0.0,8.40,Begumpet,NaN
3,2025-05-26 01:30:00,23.9,92.0,0.0,8.25,Begumpet,NaN
4,2025-05-26 02:00:00,23.8,92.0,0.0,8.10,Begumpet,NaN


In [69]:
combined_df.to_csv("weather_data.csv", index=False)

In [70]:
combined_df[combined_df['location'] == 'Kondapur']

,time,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,location,windspeed_10m
719,2025-05-26 00:00:00,23.70,90.0,0.0,11.20,Kondapur,NaN
720,2025-05-26 00:30:00,23.60,90.5,0.0,10.85,Kondapur,NaN
721,2025-05-26 01:00:00,23.50,91.0,0.0,10.50,Kondapur,NaN
722,2025-05-26 01:30:00,23.45,91.5,0.0,9.80,Kondapur,NaN
723,2025-05-26 02:00:00,23.40,92.0,0.0,9.10,Kondapur,NaN
...,...,...,...,...,...,...,...
1433,2025-06-09 21:00:00,33.40,34.0,0.0,22.70,Kondapur,NaN
1434,2025-06-09 21:30:00,32.65,37.5,0.0,24.15,Kondapur,NaN
1435,2025-06-09 22:00:00,31.90,41.0,0.0,25.60,Kondapur,NaN
1436,2025-06-09 22:30:00,31.20,44.0,0.0,26.60,Kondapur,NaN


In [71]:
combined_df[combined_df['time'] == '2025-05-29 00:00:00']

,time,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,location,windspeed_10m
144,2025-05-29,24.0,93.0,0.2,NaN,Begumpet,9.4
863,2025-05-29,24.0,93.0,0.2,NaN,Kondapur,9.4
1582,2025-05-29,24.0,93.0,0.2,NaN,Miyapur,9.4
2301,2025-05-29,24.0,93.0,0.2,NaN,Secunderabad,9.4


In [72]:
combined_df.loc[combined_df['temperature_2m'].isna(),'time'].dt.date.unique() #thank you god i love you

array([], dtype=object)

In [73]:
combined_df.loc[combined_df['relative_humidity_2m'].isna(),'time'].dt.date.unique() # uff my pancreas are relaxing

array([], dtype=object)

In [74]:
combined_df.loc[combined_df['precipitation'].isna(),'time'].dt.date.unique() # aahhhh that's my girl

array([], dtype=object)

In [75]:
combined_df.loc[combined_df['wind_speed_10m'].isna(),'time'].dt.date.unique() # ASSHOLE!

array([datetime.date(2025, 5, 27), datetime.date(2025, 5, 28),
       datetime.date(2025, 5, 29)], dtype=object)

In [76]:
combined_df['wind_speed_combined'] = combined_df[['wind_speed_10m', 'windspeed_10m']].mean(axis=1, skipna=True)

In [79]:
combined_df.drop(columns=['wind_speed_10m', 'windspeed_10m'], inplace=True)

In [77]:
combined_df.loc[combined_df['wind_speed_combined'].isna(),'time'].dt.date.unique() # ASSHOLE + all bad words in the world

array([datetime.date(2025, 5, 27), datetime.date(2025, 5, 28),
       datetime.date(2025, 5, 29)], dtype=object)

In [80]:
combined_df_bkp = combined_df.copy()

In [82]:
# Step 1: Set the datetime column as the index
combined_df = combined_df.set_index('time')

# Step 2: Interpolate using time method
combined_df['wind_speed_combined'] = combined_df['wind_speed_combined'].interpolate(method='time')

# Step 3: Reset the index if you want the time column back as a column
combined_df = combined_df.reset_index()

In [83]:
combined_df.head()

,time,temperature_2m,relative_humidity_2m,precipitation,location,wind_speed_combined
0,2025-05-26 00:00:00,24.2,91.0,0.0,Begumpet,8.80
1,2025-05-26 00:30:00,24.1,91.5,0.0,Begumpet,8.60
2,2025-05-26 01:00:00,24.0,92.0,0.0,Begumpet,8.40
3,2025-05-26 01:30:00,23.9,92.0,0.0,Begumpet,8.25
4,2025-05-26 02:00:00,23.8,92.0,0.0,Begumpet,8.10


In [84]:
combined_df.loc[combined_df['wind_speed_combined'].isna(),'time'].dt.date.unique() # I LOVE ME MUAAHHHH SAMYUUUU!!!!

array([], dtype=object)

In [85]:
combined_df.to_csv("weather_data_30.csv", index=False)

### Fetch data future

In [86]:
import requests
import pandas as pd

In [87]:
# Coordinates
locations = {
    "Kondapur": (17.4579, 78.3650),
    "Miyapur": (17.5008, 78.3568),
    "Secunderabad": (17.4399, 78.4983),
    "Begumpet": (17.4432, 78.4600)
}

In [104]:
# Variables
weather_vars = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "wind_speed_10m"
]

# Dates
archive_start = "2025-06-12"
today = "2025-06-16"
forecast_end = "2025-06-09"

# Timezone
tz = "Asia%2FKolkata"

In [105]:
def fetch_weather_today(lat, lon, location_name, date):
    # date = "2025-05-29"
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation,windspeed_10m"
        f"&start_date={date}&end_date={date}"
        f"&timezone=Asia%2FKolkata"
    )

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["hourly"])
        df["location"] = location_name
        return df
    else:
        print(f"❌ Failed for {location_name}: {response.status_code} - {response.text}")
        return None

Kondapur -> Secunderabad

In [106]:
df_konda = fetch_weather_today(locations['Kondapur'][0], locations['Kondapur'][1], "Kondapur", archive_start)

In [107]:
df_konda['time'] = pd.to_datetime(df_konda['time'])

In [108]:
df_konda

,time,temperature_2m,relative_humidity_2m,precipitation,windspeed_10m,location
0,2025-06-12 00:00:00,29.6,46,0.0,35.7,Kondapur
1,2025-06-12 01:00:00,28.6,50,0.0,33.2,Kondapur
2,2025-06-12 02:00:00,27.8,53,0.0,31.3,Kondapur
3,2025-06-12 03:00:00,27.0,57,0.0,30.4,Kondapur
4,2025-06-12 04:00:00,26.3,61,0.0,29.9,Kondapur
5,2025-06-12 05:00:00,26.1,63,0.0,29.9,Kondapur
6,2025-06-12 06:00:00,26.4,62,0.0,30.7,Kondapur
7,2025-06-12 07:00:00,27.1,59,0.0,32.3,Kondapur
8,2025-06-12 08:00:00,28.1,54,0.0,33.1,Kondapur
9,2025-06-12 09:00:00,29.5,48,0.0,32.8,Kondapur
